In [1]:
# If your PC cannot handle the large memory requirement for the .npz files, this version splits them into smaller sections, look for the other file "dataplottingsplit"
# to visualize this data

# Be sure to first have pyspedas installed first through: pip install pyspedas or update with pip install --upgrade & pip install pyspedas --upgrade
# Other packages may include: pip install pandas --upgrade & pip install statsmodels --upgrade
import csv
import pandas
import pyspedas
from pyspedas import time_double
import os
import numpy as np
import time
from multiprocessing import Pool
from scipy import stats
#from scipy.stats import mannwhitneyu
#from scipy.stats import ranksums
#from scipy.signal import medfilt

# Download the .csv file from SuperMAG at https://supermag.jhuapl.edu/substorms/ (an account is required, just make it anonymous), rename it to "substormdata.csv"
# Use the following for Linux/Mac:
#datadirectory=os.path.expanduser('~')+'/Downloads/Data/' # The ending forward slash is necessary
# Use the following for Windows:
datadirectory=os.path.join(os.path.join(os.environ['USERPROFILE']),'Downloads\\Data\\') # The two ending back slashes are necessary

# It may be necessary to use a conversion factor on some of these parameters, example:
tempconversionfactor=8.61732814974493*1e-5 # for K to eV in temperature

substormfile=datadirectory+'substormdata.csv'
with open(substormfile,'r') as csv_file:
    csv_reader=csv.reader(csv_file)
substormdata=pandas.read_csv(substormfile)
substormtime=substormdata.iloc[0:len(substormdata),0]
subtrange=time_double(substormtime)

# Requires the use of omnisave.ipynb first
# For more info on these parameters, visit https://cdaweb.gsfc.nasa.gov/misc/NotesO.html#OMNI_HRO2_1MIN
loadedarrays=np.load(datadirectory+'myomnidata.npz')
bxtimes=loadedarrays['arr_0']
#imfvalues=loadedarrays['arr_1']
#plsvalues=loadedarrays['arr_2']
#imfptsvalues=loadedarrays['arr_3']
#plsptsvalues=loadedarrays['arr_4']
#percinterpvalues=loadedarrays['arr_5']
#timeshiftvalues=loadedarrays['arr_6']
#rmstimeshiftvalues=loadedarrays['arr_7']
#rmsphasevalues=loadedarrays['arr_8']
#timebtwnobsvalues=loadedarrays['arr_9']
#bmagvalues=loadedarrays['arr_10']
bxvalues=loadedarrays['arr_11']
byvalues=loadedarrays['arr_12']
bzvalues=loadedarrays['arr_13']
bygsmvalues=loadedarrays['arr_14']
bzgsmvalues=loadedarrays['arr_15']
#rmssdbvalues=loadedarrays['arr_16']
#rmssdfldvecvalues=loadedarrays['arr_17']
#speedvalues=loadedarrays['arr_18']
vxvalues=loadedarrays['arr_19']
vyvalues=loadedarrays['arr_20']
vzvalues=loadedarrays['arr_21']
#protondenvalues=loadedarrays['arr_22']
#temperaturevalues=loadedarrays['arr_23']*tempconversionfactor
#alpharatiovalues=loadedarrays['arr_24']
#flowpressurevalues=loadedarrays['arr_25']
efieldmeasuredvalues=loadedarrays['arr_26']
#betavalues=loadedarrays['arr_27']
#alfmachvalues=loadedarrays['arr_28']
#magmachvalues=loadedarrays['arr_29']
#xposvalues=loadedarrays['arr_30']
#yposvalues=loadedarrays['arr_31']
#zposvalues=loadedarrays['arr_32']
#bowshockxvalues=loadedarrays['arr_33']
#bowshockyvalues=loadedarrays['arr_34']
#bowshockzvalues=loadedarrays['arr_35']
#aeindexvalues=loadedarrays['arr_36']
#alindexvalues=loadedarrays['arr_37']
#auindexvalues=loadedarrays['arr_38']
#symdvalues=loadedarrays['arr_39']
#symhvalues=loadedarrays['arr_40']
#asydvalues=loadedarrays['arr_41']
#asyhvalues=loadedarrays['arr_42']

# Add any derived parameters, take note of units
efieldxgsecalculatedvalues=1e3*1e3*1e-9*(vzvalues*byvalues-vyvalues*bzvalues) # values are in mV/m
efieldygsecalculatedvalues=1e3*1e3*1e-9*(vxvalues*bzvalues-vzvalues*bxvalues) # values are in mV/m
efieldzgsecalculatedvalues=1e3*1e3*1e-9*(vyvalues*bxvalues-vxvalues*byvalues) # values are in mV/m
efieldcalculatedvalues=np.sqrt(efieldxgsecalculatedvalues**2+efieldygsecalculatedvalues**2+efieldzgsecalculatedvalues**2) # values are in mV/m
#magpressurevalues=1e9*(bmagvalues*1e-9)**2/(2*1.256637062*1e-6) # values are in nPa
#alfvenspeedvalues=1e-3*1e-9*bmagvalues/np.sqrt(1.256637062*1e-6*1e6*protondenvalues*1.672621923*1e-27*(1+alpharatiovalues)) # values are in km/s
#entropyvalues=temperaturevalues*(protondenvalues)**(-2/3) # values are in eV*cm^2
#imfconeanglevalues=np.arccos(bxvalues/bmagvalues)*180/np.pi # converted to degrees written in python as \N{DEGREE SIGN}
# The following below is an attempt to make the clock angle useable, unfortunately it still jumps from 0 to 360, and I don't know what to do about that
#for i in range(len(bygsmvalues)):
#    if bygsmvalues[i]>=0 and bzgsmvalues[i]>=0:
#        imfclockanglevalues[i]=np.arctan(np.abs(bygsmvalues[i]/bzgsmvalues[i]))*180/np.pi # converted to degrees written in python as \N{DEGREE SIGN}
#    if bygsmvalues[i]>=0 and bzgsmvalues[i]<0:
#        imfclockanglevalues[i]=np.arctan(np.abs(bzgsmvalues[i]/bygsmvalues[i]))*180/np.pi+90
#    if bygsmvalues[i]<0 and bzgsmvalues[i]<0:
#        imfclockanglevalues[i]=np.arctan(np.abs(bygsmvalues[i]/bzgsmvalues[i]))*180/np.pi+180
#    if bygsmvalues[i]<0 and bzgsmvalues[i]>=0:
#        imfclockanglevalues[i]=np.arctan(np.abs(bzgsmvalues[i]/bygsmvalues[i]))*180/np.pi+270
#    if bygsmvalues[i]==np.nan:
#        imfclockanglevalues[i]=np.nan
#    if bzgsmvalues[i]==np.nan:
#        imfclockanglevalues[i]=np.nan
#poyntingvalues=1e3*(1/(1.256637062*1e-6))*1e3*1e-18*speedvalues*bmagvalues**2 # values are in mW/m^2
#epsilonvalues=1e-3*1e-9*poyntingvalues*4*np.pi*(7*6.3781*1e6)**2*np.sin(np.arctan(bygsmvalues/bzgsmvalues)/2)**4 # values are in GW

In [2]:
# This is including the cutoff time for the minimum time interval between substorms, and this is comparing onset with its immediate surrounding times

# the data as different files, this is an old script that I used for when the file sizes were too big, but it was a coding error that was the main culprit on why they were so big.
# I recommend running the entire jupyter document at once to save time

# Select parameter & time interval to look at:
paramvalues=byvalues
filename='byvalues'

start=-120 # Negative means before substorm, values in minutes
end=-90
substormcutoff=6*60 # For the minimum time in-between substorms i.e. there must be a period of at least x minutes before an onset event that contains no other onsets
averagestart=-5 # To calculte the total background averages, a range must be given i.e. a range between 60 and 30 minutes before an onset would be -60 and -30
averageend=5

# !!!WARNING!!!  This process is sped up via multiprocessing on all CPU cores, may slow down PC as a result  !!!WARNING!!!

starttime=time.time()
varrange=range(start,end+1)

subindex=bxtimes.searchsorted(subtrange) # Finds the index of each starting time. This is such a neat trick that should be documented more on, greatly reduces calculation time
subindex=list(subindex)
for i in reversed(range(len(subindex)-1)):
    if subindex[i]-subindex[i-1]<substormcutoff: # Value again in minutes
        del subindex[i]
subindex=np.concatenate((574,subindex),axis=None) # The first value needs to be replaced here

#paramvalues[paramvalues==0]=np.nan # Some variables that have many points measured at exactly 0 should actually be nan values
#paramvalues[paramvalues==np.inf]=np.nan # Same as above but for inf values, usually occur when dividing by 0
#paramvalues[paramvalues==999999]=np.nan # Other files may use this as the default for no value

paramvaluesgen=[]
for i in subindex:
    for j in range(averagestart,averageend+1):
        paramvaluesgen.append(paramvalues[i+j])
genmean=np.array(np.nanmean(paramvaluesgen)) # These must also be converted in order to be saved as an array by np.savez
genvariance=np.array(np.nanvar(paramvaluesgen))

paramvaluesgen=np.array(paramvaluesgen)
paramvaluesgen=np.nanmean(paramvaluesgen.reshape(-1,(abs(averageend-averagestart)+1)),axis=1) # Takes the average of each time period before a substorm onset
paramvaluesgen=paramvaluesgen.astype('float') # Only works with float values apparently

ttest=[]
tpvalues=[]
means=[]
variances=[]
importantvalues=[]
for i in varrange:
    delayedsubindex=subindex+i
    paramimportant=[]
    for i in delayedsubindex:
        paramimportant.append(paramvalues[i])
    importantvalues.append(paramimportant)
    paramimportant=np.array(paramimportant) # The test requires this apparently
    ttestvalues=stats.ttest_ind(a=paramvaluesgen[~np.isnan(paramvaluesgen)],b=paramimportant[~np.isnan(paramimportant)],equal_var=False) # The variations are NOT equal, so this is a Welch Test
    ttest.append(np.abs(ttestvalues.statistic))
    tpvalues.append(ttestvalues.pvalue)
    means.append(np.nanmean(paramimportant))
    variances.append(np.nanvar(paramimportant))

print("--- Time To Calculate: "+str(time.time()-starttime)+" seconds ("+str((time.time()-starttime)/60)+" minutes) ---")
print('Percent of substorms that fit the range criteria: '+str(len(subindex)/len(subtrange)*100)+' %')

starttime=time.time()
#np.savez(datadirectory+str(filename)+'start'+str(start)+'end'+str(end)+'nointerval'+'processeddata.npz',ttest,tpvalues,means,variances,genmean,genvariance,kstest,kspvalues,cvtest,cvpvalues,rstest,rspvalues,mwtest,mwpvalues)
# Comment out below if including the above terms
np.savez(datadirectory+str(filename)+'start'+str(start)+'end'+str(end)+'nointerval'+str(substormcutoff)+str(averagestart)+str(averageend)+'processeddata.npz',ttest,tpvalues,means,variances,genmean,genvariance,importantvalues)
#np.savez(datadirectory+str(filename)+'start'+str(start)+'end'+str(end)+'nointerval'+str(substormcutoff)+str(averagestart)+str(averageend)+'processeddata.npz',ttest,tpvalues,means,variances,genmean,genvariance)
print("--- Time To Save File: "+str(time.time()-starttime)+" seconds ("+str((time.time()-starttime)/60)+" minutes) ---")

22-Oct-24 18:49:06: C:\Users\getma\AppData\Local\Temp\ipykernel_12620\2748658978.py:40: RuntimeWarning: Mean of empty slice
  paramvaluesgen=np.nanmean(paramvaluesgen.reshape(-1,(abs(averageend-averagestart)+1)),axis=1) # Takes the average of each time period before a substorm onset



--- Time To Calculate: 0.1779932975769043 seconds (0.0029665549596150716 minutes) ---
Percent of substorms that fit the range criteria: 20.44747772003423 %
--- Time To Save File: 0.011999845504760742 seconds (0.0001999974250793457 minutes) ---


In [3]:
start=-89 # Negative means before substorm, values in minutes
end=-60
substormcutoff=6*60 # For the minimum time in-between substorms i.e. there must be a period of at least x minutes before an onset event that contains no other onsets
averagestart=-5 # To calculte the total background averages, a range must be given i.e. a range between 60 and 30 minutes before an onset would be -60 and -30
averageend=5

starttime=time.time()
varrange=range(start,end+1)

subindex=bxtimes.searchsorted(subtrange) # Finds the index of each starting time. This is such a neat trick that should be documented more on, greatly reduces calculation time
subindex=list(subindex)
for i in reversed(range(len(subindex)-1)):
    if subindex[i]-subindex[i-1]<substormcutoff: # Value again in minutes
        del subindex[i]
subindex=np.concatenate((574,subindex),axis=None) # The first value needs to be replaced here

#paramvalues[paramvalues==0]=np.nan # Some variables that have many points measured at exactly 0 should actually be nan values
#paramvalues[paramvalues==np.inf]=np.nan # Same as above but for inf values, usually occur when dividing by 0
#paramvalues[paramvalues==999999]=np.nan # Other files may use this as the default for no value

paramvaluesgen=[]
for i in subindex:
    for j in range(averagestart,averageend+1):
        paramvaluesgen.append(paramvalues[i+j])
genmean=np.array(np.nanmean(paramvaluesgen)) # These must also be converted in order to be saved as an array by np.savez
genvariance=np.array(np.nanvar(paramvaluesgen))

paramvaluesgen=np.array(paramvaluesgen)
paramvaluesgen=np.nanmean(paramvaluesgen.reshape(-1,(abs(averageend-averagestart)+1)),axis=1) # Takes the average of each time period before a substorm onset
paramvaluesgen=paramvaluesgen.astype('float') # Only works with float values apparently

ttest=[]
tpvalues=[]
means=[]
variances=[]
importantvalues=[]
for i in varrange:
    delayedsubindex=subindex+i
    paramimportant=[]
    for i in delayedsubindex:
        paramimportant.append(paramvalues[i])
    importantvalues.append(paramimportant)
    paramimportant=np.array(paramimportant) # The test requires this apparently
    ttestvalues=stats.ttest_ind(a=paramvaluesgen[~np.isnan(paramvaluesgen)],b=paramimportant[~np.isnan(paramimportant)],equal_var=False) # The variations are NOT equal, so this is a Welch Test
    ttest.append(np.abs(ttestvalues.statistic))
    tpvalues.append(ttestvalues.pvalue)
    means.append(np.nanmean(paramimportant))
    variances.append(np.nanvar(paramimportant))

print("--- Time To Calculate: "+str(time.time()-starttime)+" seconds ("+str((time.time()-starttime)/60)+" minutes) ---")
print('Percent of substorms that fit the range criteria: '+str(len(subindex)/len(subtrange)*100)+' %')

starttime=time.time()
#np.savez(datadirectory+str(filename)+'start'+str(start)+'end'+str(end)+'nointerval'+'processeddata.npz',ttest,tpvalues,means,variances,genmean,genvariance,kstest,kspvalues,cvtest,cvpvalues,rstest,rspvalues,mwtest,mwpvalues)
# Comment out below if including the above terms
np.savez(datadirectory+str(filename)+'start'+str(start)+'end'+str(end)+'nointerval'+str(substormcutoff)+str(averagestart)+str(averageend)+'processeddata.npz',ttest,tpvalues,means,variances,genmean,genvariance,importantvalues)
#np.savez(datadirectory+str(filename)+'start'+str(start)+'end'+str(end)+'nointerval'+str(substormcutoff)+str(averagestart)+str(averageend)+'processeddata.npz',ttest,tpvalues,means,variances,genmean,genvariance)
print("--- Time To Save File: "+str(time.time()-starttime)+" seconds ("+str((time.time()-starttime)/60)+" minutes) ---")

22-Oct-24 18:49:07: C:\Users\getma\AppData\Local\Temp\ipykernel_12620\4109715545.py:29: RuntimeWarning: Mean of empty slice
  paramvaluesgen=np.nanmean(paramvaluesgen.reshape(-1,(abs(averageend-averagestart)+1)),axis=1) # Takes the average of each time period before a substorm onset



--- Time To Calculate: 0.15399813652038574 seconds (0.0025666356086730957 minutes) ---
Percent of substorms that fit the range criteria: 20.44747772003423 %
--- Time To Save File: 0.012000799179077148 seconds (0.0002000133196512858 minutes) ---


In [4]:
start=-59 # Negative means before substorm, values in minutes
end=-30
substormcutoff=6*60 # For the minimum time in-between substorms i.e. there must be a period of at least x minutes before an onset event that contains no other onsets
averagestart=-5 # To calculte the total background averages, a range must be given i.e. a range between 60 and 30 minutes before an onset would be -60 and -30
averageend=5

starttime=time.time()
varrange=range(start,end+1)

subindex=bxtimes.searchsorted(subtrange) # Finds the index of each starting time. This is such a neat trick that should be documented more on, greatly reduces calculation time
subindex=list(subindex)
for i in reversed(range(len(subindex)-1)):
    if subindex[i]-subindex[i-1]<substormcutoff: # Value again in minutes
        del subindex[i]
subindex=np.concatenate((574,subindex),axis=None) # The first value needs to be replaced here

#paramvalues[paramvalues==0]=np.nan # Some variables that have many points measured at exactly 0 should actually be nan values
#paramvalues[paramvalues==np.inf]=np.nan # Same as above but for inf values, usually occur when dividing by 0
#paramvalues[paramvalues==999999]=np.nan # Other files may use this as the default for no value

paramvaluesgen=[]
for i in subindex:
    for j in range(averagestart,averageend+1):
        paramvaluesgen.append(paramvalues[i+j])
genmean=np.array(np.nanmean(paramvaluesgen)) # These must also be converted in order to be saved as an array by np.savez
genvariance=np.array(np.nanvar(paramvaluesgen))

paramvaluesgen=np.array(paramvaluesgen)
paramvaluesgen=np.nanmean(paramvaluesgen.reshape(-1,(abs(averageend-averagestart)+1)),axis=1) # Takes the average of each time period before a substorm onset
paramvaluesgen=paramvaluesgen.astype('float') # Only works with float values apparently

ttest=[]
tpvalues=[]
means=[]
variances=[]
importantvalues=[]
for i in varrange:
    delayedsubindex=subindex+i
    paramimportant=[]
    for i in delayedsubindex:
        paramimportant.append(paramvalues[i])
    importantvalues.append(paramimportant)
    paramimportant=np.array(paramimportant) # The test requires this apparently
    ttestvalues=stats.ttest_ind(a=paramvaluesgen[~np.isnan(paramvaluesgen)],b=paramimportant[~np.isnan(paramimportant)],equal_var=False) # The variations are NOT equal, so this is a Welch Test
    ttest.append(np.abs(ttestvalues.statistic))
    tpvalues.append(ttestvalues.pvalue)
    means.append(np.nanmean(paramimportant))
    variances.append(np.nanvar(paramimportant))

print("--- Time To Calculate: "+str(time.time()-starttime)+" seconds ("+str((time.time()-starttime)/60)+" minutes) ---")
print('Percent of substorms that fit the range criteria: '+str(len(subindex)/len(subtrange)*100)+' %')

starttime=time.time()
#np.savez(datadirectory+str(filename)+'start'+str(start)+'end'+str(end)+'nointerval'+'processeddata.npz',ttest,tpvalues,means,variances,genmean,genvariance,kstest,kspvalues,cvtest,cvpvalues,rstest,rspvalues,mwtest,mwpvalues)
# Comment out below if including the above terms
np.savez(datadirectory+str(filename)+'start'+str(start)+'end'+str(end)+'nointerval'+str(substormcutoff)+str(averagestart)+str(averageend)+'processeddata.npz',ttest,tpvalues,means,variances,genmean,genvariance,importantvalues)
#np.savez(datadirectory+str(filename)+'start'+str(start)+'end'+str(end)+'nointerval'+str(substormcutoff)+str(averagestart)+str(averageend)+'processeddata.npz',ttest,tpvalues,means,variances,genmean,genvariance)
print("--- Time To Save File: "+str(time.time()-starttime)+" seconds ("+str((time.time()-starttime)/60)+" minutes) ---")

22-Oct-24 18:49:07: C:\Users\getma\AppData\Local\Temp\ipykernel_12620\1767348083.py:29: RuntimeWarning: Mean of empty slice
  paramvaluesgen=np.nanmean(paramvaluesgen.reshape(-1,(abs(averageend-averagestart)+1)),axis=1) # Takes the average of each time period before a substorm onset



--- Time To Calculate: 0.1549975872039795 seconds (0.002583293120066325 minutes) ---
Percent of substorms that fit the range criteria: 20.44747772003423 %
--- Time To Save File: 0.011997222900390625 seconds (0.0001999537150065104 minutes) ---


In [5]:
start=-29 # Negative means before substorm, values in minutes
end=0
substormcutoff=6*60 # For the minimum time in-between substorms i.e. there must be a period of at least x minutes before an onset event that contains no other onsets
averagestart=-5 # To calculte the total background averages, a range must be given i.e. a range between 60 and 30 minutes before an onset would be -60 and -30
averageend=5

starttime=time.time()
varrange=range(start,end+1)

subindex=bxtimes.searchsorted(subtrange) # Finds the index of each starting time. This is such a neat trick that should be documented more on, greatly reduces calculation time
subindex=list(subindex)
for i in reversed(range(len(subindex)-1)):
    if subindex[i]-subindex[i-1]<substormcutoff: # Value again in minutes
        del subindex[i]
subindex=np.concatenate((574,subindex),axis=None) # The first value needs to be replaced here

#paramvalues[paramvalues==0]=np.nan # Some variables that have many points measured at exactly 0 should actually be nan values
#paramvalues[paramvalues==np.inf]=np.nan # Same as above but for inf values, usually occur when dividing by 0
#paramvalues[paramvalues==999999]=np.nan # Other files may use this as the default for no value

paramvaluesgen=[]
for i in subindex:
    for j in range(averagestart,averageend+1):
        paramvaluesgen.append(paramvalues[i+j])
genmean=np.array(np.nanmean(paramvaluesgen)) # These must also be converted in order to be saved as an array by np.savez
genvariance=np.array(np.nanvar(paramvaluesgen))

paramvaluesgen=np.array(paramvaluesgen)
paramvaluesgen=np.nanmean(paramvaluesgen.reshape(-1,(abs(averageend-averagestart)+1)),axis=1) # Takes the average of each time period before a substorm onset
paramvaluesgen=paramvaluesgen.astype('float') # Only works with float values apparently

ttest=[]
tpvalues=[]
means=[]
variances=[]
importantvalues=[]
for i in varrange:
    delayedsubindex=subindex+i
    paramimportant=[]
    for i in delayedsubindex:
        paramimportant.append(paramvalues[i])
    importantvalues.append(paramimportant)
    paramimportant=np.array(paramimportant) # The test requires this apparently
    ttestvalues=stats.ttest_ind(a=paramvaluesgen[~np.isnan(paramvaluesgen)],b=paramimportant[~np.isnan(paramimportant)],equal_var=False) # The variations are NOT equal, so this is a Welch Test
    ttest.append(np.abs(ttestvalues.statistic))
    tpvalues.append(ttestvalues.pvalue)
    means.append(np.nanmean(paramimportant))
    variances.append(np.nanvar(paramimportant))

print("--- Time To Calculate: "+str(time.time()-starttime)+" seconds ("+str((time.time()-starttime)/60)+" minutes) ---")
print('Percent of substorms that fit the range criteria: '+str(len(subindex)/len(subtrange)*100)+' %')

starttime=time.time()
#np.savez(datadirectory+str(filename)+'start'+str(start)+'end'+str(end)+'nointerval'+'processeddata.npz',ttest,tpvalues,means,variances,genmean,genvariance,kstest,kspvalues,cvtest,cvpvalues,rstest,rspvalues,mwtest,mwpvalues)
# Comment out below if including the above terms
np.savez(datadirectory+str(filename)+'start'+str(start)+'end'+str(end)+'nointerval'+str(substormcutoff)+str(averagestart)+str(averageend)+'processeddata.npz',ttest,tpvalues,means,variances,genmean,genvariance,importantvalues)
#np.savez(datadirectory+str(filename)+'start'+str(start)+'end'+str(end)+'nointerval'+str(substormcutoff)+str(averagestart)+str(averageend)+'processeddata.npz',ttest,tpvalues,means,variances,genmean,genvariance)
print("--- Time To Save File: "+str(time.time()-starttime)+" seconds ("+str((time.time()-starttime)/60)+" minutes) ---")

22-Oct-24 18:49:07: C:\Users\getma\AppData\Local\Temp\ipykernel_12620\1580163413.py:29: RuntimeWarning: Mean of empty slice
  paramvaluesgen=np.nanmean(paramvaluesgen.reshape(-1,(abs(averageend-averagestart)+1)),axis=1) # Takes the average of each time period before a substorm onset



--- Time To Calculate: 0.15499448776245117 seconds (0.0025832414627075194 minutes) ---
Percent of substorms that fit the range criteria: 20.44747772003423 %
--- Time To Save File: 0.012000322341918945 seconds (0.00020000537236531575 minutes) ---


In [6]:
start=1 # Negative means before substorm, values in minutes
end=30
substormcutoff=6*60 # For the minimum time in-between substorms i.e. there must be a period of at least x minutes before an onset event that contains no other onsets
averagestart=-5 # To calculte the total background averages, a range must be given i.e. a range between 60 and 30 minutes before an onset would be -60 and -30
averageend=5

starttime=time.time()
varrange=range(start,end+1)

subindex=bxtimes.searchsorted(subtrange) # Finds the index of each starting time. This is such a neat trick that should be documented more on, greatly reduces calculation time
subindex=list(subindex)
for i in reversed(range(len(subindex)-1)):
    if subindex[i]-subindex[i-1]<substormcutoff: # Value again in minutes
        del subindex[i]
subindex=np.concatenate((574,subindex),axis=None) # The first value needs to be replaced here

#paramvalues[paramvalues==0]=np.nan # Some variables that have many points measured at exactly 0 should actually be nan values
#paramvalues[paramvalues==np.inf]=np.nan # Same as above but for inf values, usually occur when dividing by 0
#paramvalues[paramvalues==999999]=np.nan # Other files may use this as the default for no value

paramvaluesgen=[]
for i in subindex:
    for j in range(averagestart,averageend+1):
        paramvaluesgen.append(paramvalues[i+j])
genmean=np.array(np.nanmean(paramvaluesgen)) # These must also be converted in order to be saved as an array by np.savez
genvariance=np.array(np.nanvar(paramvaluesgen))

paramvaluesgen=np.array(paramvaluesgen)
paramvaluesgen=np.nanmean(paramvaluesgen.reshape(-1,(abs(averageend-averagestart)+1)),axis=1) # Takes the average of each time period before a substorm onset
paramvaluesgen=paramvaluesgen.astype('float') # Only works with float values apparently

ttest=[]
tpvalues=[]
means=[]
variances=[]
importantvalues=[]
for i in varrange:
    delayedsubindex=subindex+i
    paramimportant=[]
    for i in delayedsubindex:
        paramimportant.append(paramvalues[i])
    importantvalues.append(paramimportant)
    paramimportant=np.array(paramimportant) # The test requires this apparently
    ttestvalues=stats.ttest_ind(a=paramvaluesgen[~np.isnan(paramvaluesgen)],b=paramimportant[~np.isnan(paramimportant)],equal_var=False) # The variations are NOT equal, so this is a Welch Test
    ttest.append(np.abs(ttestvalues.statistic))
    tpvalues.append(ttestvalues.pvalue)
    means.append(np.nanmean(paramimportant))
    variances.append(np.nanvar(paramimportant))

print("--- Time To Calculate: "+str(time.time()-starttime)+" seconds ("+str((time.time()-starttime)/60)+" minutes) ---")
print('Percent of substorms that fit the range criteria: '+str(len(subindex)/len(subtrange)*100)+' %')

starttime=time.time()
#np.savez(datadirectory+str(filename)+'start'+str(start)+'end'+str(end)+'nointerval'+'processeddata.npz',ttest,tpvalues,means,variances,genmean,genvariance,kstest,kspvalues,cvtest,cvpvalues,rstest,rspvalues,mwtest,mwpvalues)
# Comment out below if including the above terms
np.savez(datadirectory+str(filename)+'start'+str(start)+'end'+str(end)+'nointerval'+str(substormcutoff)+str(averagestart)+str(averageend)+'processeddata.npz',ttest,tpvalues,means,variances,genmean,genvariance,importantvalues)
#np.savez(datadirectory+str(filename)+'start'+str(start)+'end'+str(end)+'nointerval'+str(substormcutoff)+str(averagestart)+str(averageend)+'processeddata.npz',ttest,tpvalues,means,variances,genmean,genvariance)
print("--- Time To Save File: "+str(time.time()-starttime)+" seconds ("+str((time.time()-starttime)/60)+" minutes) ---")

22-Oct-24 18:49:07: C:\Users\getma\AppData\Local\Temp\ipykernel_12620\4270648705.py:29: RuntimeWarning: Mean of empty slice
  paramvaluesgen=np.nanmean(paramvaluesgen.reshape(-1,(abs(averageend-averagestart)+1)),axis=1) # Takes the average of each time period before a substorm onset



--- Time To Calculate: 0.15599846839904785 seconds (0.0025999744733174643 minutes) ---
Percent of substorms that fit the range criteria: 20.44747772003423 %
--- Time To Save File: 0.04699850082397461 seconds (0.0007833083470662435 minutes) ---


In [7]:
start=31 # Negative means before substorm, values in minutes
end=60
substormcutoff=6*60 # For the minimum time in-between substorms i.e. there must be a period of at least x minutes before an onset event that contains no other onsets
averagestart=-5 # To calculte the total background averages, a range must be given i.e. a range between 60 and 30 minutes before an onset would be -60 and -30
averageend=5

starttime=time.time()
varrange=range(start,end+1)

subindex=bxtimes.searchsorted(subtrange) # Finds the index of each starting time. This is such a neat trick that should be documented more on, greatly reduces calculation time
subindex=list(subindex)
for i in reversed(range(len(subindex)-1)):
    if subindex[i]-subindex[i-1]<substormcutoff: # Value again in minutes
        del subindex[i]
subindex=np.concatenate((574,subindex),axis=None) # The first value needs to be replaced here

#paramvalues[paramvalues==0]=np.nan # Some variables that have many points measured at exactly 0 should actually be nan values
#paramvalues[paramvalues==np.inf]=np.nan # Same as above but for inf values, usually occur when dividing by 0
#paramvalues[paramvalues==999999]=np.nan # Other files may use this as the default for no value

paramvaluesgen=[]
for i in subindex:
    for j in range(averagestart,averageend+1):
        paramvaluesgen.append(paramvalues[i+j])
genmean=np.array(np.nanmean(paramvaluesgen)) # These must also be converted in order to be saved as an array by np.savez
genvariance=np.array(np.nanvar(paramvaluesgen))

paramvaluesgen=np.array(paramvaluesgen)
paramvaluesgen=np.nanmean(paramvaluesgen.reshape(-1,(abs(averageend-averagestart)+1)),axis=1) # Takes the average of each time period before a substorm onset
paramvaluesgen=paramvaluesgen.astype('float') # Only works with float values apparently

ttest=[]
tpvalues=[]
means=[]
variances=[]
importantvalues=[]
for i in varrange:
    delayedsubindex=subindex+i
    paramimportant=[]
    for i in delayedsubindex:
        paramimportant.append(paramvalues[i])
    importantvalues.append(paramimportant)
    paramimportant=np.array(paramimportant) # The test requires this apparently
    ttestvalues=stats.ttest_ind(a=paramvaluesgen[~np.isnan(paramvaluesgen)],b=paramimportant[~np.isnan(paramimportant)],equal_var=False) # The variations are NOT equal, so this is a Welch Test
    ttest.append(np.abs(ttestvalues.statistic))
    tpvalues.append(ttestvalues.pvalue)
    means.append(np.nanmean(paramimportant))
    variances.append(np.nanvar(paramimportant))

print("--- Time To Calculate: "+str(time.time()-starttime)+" seconds ("+str((time.time()-starttime)/60)+" minutes) ---")
print('Percent of substorms that fit the range criteria: '+str(len(subindex)/len(subtrange)*100)+' %')

starttime=time.time()
#np.savez(datadirectory+str(filename)+'start'+str(start)+'end'+str(end)+'nointerval'+'processeddata.npz',ttest,tpvalues,means,variances,genmean,genvariance,kstest,kspvalues,cvtest,cvpvalues,rstest,rspvalues,mwtest,mwpvalues)
# Comment out below if including the above terms
np.savez(datadirectory+str(filename)+'start'+str(start)+'end'+str(end)+'nointerval'+str(substormcutoff)+str(averagestart)+str(averageend)+'processeddata.npz',ttest,tpvalues,means,variances,genmean,genvariance,importantvalues)
#np.savez(datadirectory+str(filename)+'start'+str(start)+'end'+str(end)+'nointerval'+str(substormcutoff)+str(averagestart)+str(averageend)+'processeddata.npz',ttest,tpvalues,means,variances,genmean,genvariance)
print("--- Time To Save File: "+str(time.time()-starttime)+" seconds ("+str((time.time()-starttime)/60)+" minutes) ---")

22-Oct-24 18:49:07: C:\Users\getma\AppData\Local\Temp\ipykernel_12620\713931069.py:29: RuntimeWarning: Mean of empty slice
  paramvaluesgen=np.nanmean(paramvaluesgen.reshape(-1,(abs(averageend-averagestart)+1)),axis=1) # Takes the average of each time period before a substorm onset



--- Time To Calculate: 0.15400242805480957 seconds (0.002566707134246826 minutes) ---
Percent of substorms that fit the range criteria: 20.44747772003423 %
--- Time To Save File: 0.027995824813842773 seconds (0.0004665970802307129 minutes) ---


In [8]:
start=61 # Negative means before substorm, values in minutes
end=90
substormcutoff=6*60 # For the minimum time in-between substorms i.e. there must be a period of at least x minutes before an onset event that contains no other onsets
averagestart=-5 # To calculte the total background averages, a range must be given i.e. a range between 60 and 30 minutes before an onset would be -60 and -30
averageend=5

starttime=time.time()
varrange=range(start,end+1)

subindex=bxtimes.searchsorted(subtrange) # Finds the index of each starting time. This is such a neat trick that should be documented more on, greatly reduces calculation time
subindex=list(subindex)
for i in reversed(range(len(subindex)-1)):
    if subindex[i]-subindex[i-1]<substormcutoff: # Value again in minutes
        del subindex[i]
subindex=np.concatenate((574,subindex),axis=None) # The first value needs to be replaced here

#paramvalues[paramvalues==0]=np.nan # Some variables that have many points measured at exactly 0 should actually be nan values
#paramvalues[paramvalues==np.inf]=np.nan # Same as above but for inf values, usually occur when dividing by 0
#paramvalues[paramvalues==999999]=np.nan # Other files may use this as the default for no value

paramvaluesgen=[]
for i in subindex:
    for j in range(averagestart,averageend+1):
        paramvaluesgen.append(paramvalues[i+j])
genmean=np.array(np.nanmean(paramvaluesgen)) # These must also be converted in order to be saved as an array by np.savez
genvariance=np.array(np.nanvar(paramvaluesgen))

paramvaluesgen=np.array(paramvaluesgen)
paramvaluesgen=np.nanmean(paramvaluesgen.reshape(-1,(abs(averageend-averagestart)+1)),axis=1) # Takes the average of each time period before a substorm onset
paramvaluesgen=paramvaluesgen.astype('float') # Only works with float values apparently

ttest=[]
tpvalues=[]
means=[]
variances=[]
importantvalues=[]
for i in varrange:
    delayedsubindex=subindex+i
    paramimportant=[]
    for i in delayedsubindex:
        paramimportant.append(paramvalues[i])
    importantvalues.append(paramimportant)
    paramimportant=np.array(paramimportant) # The test requires this apparently
    ttestvalues=stats.ttest_ind(a=paramvaluesgen[~np.isnan(paramvaluesgen)],b=paramimportant[~np.isnan(paramimportant)],equal_var=False) # The variations are NOT equal, so this is a Welch Test
    ttest.append(np.abs(ttestvalues.statistic))
    tpvalues.append(ttestvalues.pvalue)
    means.append(np.nanmean(paramimportant))
    variances.append(np.nanvar(paramimportant))

print("--- Time To Calculate: "+str(time.time()-starttime)+" seconds ("+str((time.time()-starttime)/60)+" minutes) ---")
print('Percent of substorms that fit the range criteria: '+str(len(subindex)/len(subtrange)*100)+' %')

starttime=time.time()
#np.savez(datadirectory+str(filename)+'start'+str(start)+'end'+str(end)+'nointerval'+'processeddata.npz',ttest,tpvalues,means,variances,genmean,genvariance,kstest,kspvalues,cvtest,cvpvalues,rstest,rspvalues,mwtest,mwpvalues)
# Comment out below if including the above terms
np.savez(datadirectory+str(filename)+'start'+str(start)+'end'+str(end)+'nointerval'+str(substormcutoff)+str(averagestart)+str(averageend)+'processeddata.npz',ttest,tpvalues,means,variances,genmean,genvariance,importantvalues)
#np.savez(datadirectory+str(filename)+'start'+str(start)+'end'+str(end)+'nointerval'+str(substormcutoff)+str(averagestart)+str(averageend)+'processeddata.npz',ttest,tpvalues,means,variances,genmean,genvariance)
print("--- Time To Save File: "+str(time.time()-starttime)+" seconds ("+str((time.time()-starttime)/60)+" minutes) ---")

22-Oct-24 18:49:08: C:\Users\getma\AppData\Local\Temp\ipykernel_12620\756379360.py:29: RuntimeWarning: Mean of empty slice
  paramvaluesgen=np.nanmean(paramvaluesgen.reshape(-1,(abs(averageend-averagestart)+1)),axis=1) # Takes the average of each time period before a substorm onset



--- Time To Calculate: 0.15599679946899414 seconds (0.002599946657816569 minutes) ---
Percent of substorms that fit the range criteria: 20.44747772003423 %
--- Time To Save File: 0.02700066566467285 seconds (0.0004500110944112142 minutes) ---


In [9]:
start=91 # Negative means before substorm, values in minutes
end=120
substormcutoff=6*60 # For the minimum time in-between substorms i.e. there must be a period of at least x minutes before an onset event that contains no other onsets
averagestart=-5 # To calculte the total background averages, a range must be given i.e. a range between 60 and 30 minutes before an onset would be -60 and -30
averageend=5

starttime=time.time()
varrange=range(start,end+1)

subindex=bxtimes.searchsorted(subtrange) # Finds the index of each starting time. This is such a neat trick that should be documented more on, greatly reduces calculation time
subindex=list(subindex)
for i in reversed(range(len(subindex)-1)):
    if subindex[i]-subindex[i-1]<substormcutoff: # Value again in minutes
        del subindex[i]
subindex=np.concatenate((574,subindex),axis=None) # The first value needs to be replaced here

#paramvalues[paramvalues==0]=np.nan # Some variables that have many points measured at exactly 0 should actually be nan values
#paramvalues[paramvalues==np.inf]=np.nan # Same as above but for inf values, usually occur when dividing by 0
#paramvalues[paramvalues==999999]=np.nan # Other files may use this as the default for no value

paramvaluesgen=[]
for i in subindex:
    for j in range(averagestart,averageend+1):
        paramvaluesgen.append(paramvalues[i+j])
genmean=np.array(np.nanmean(paramvaluesgen)) # These must also be converted in order to be saved as an array by np.savez
genvariance=np.array(np.nanvar(paramvaluesgen))

paramvaluesgen=np.array(paramvaluesgen)
paramvaluesgen=np.nanmean(paramvaluesgen.reshape(-1,(abs(averageend-averagestart)+1)),axis=1) # Takes the average of each time period before a substorm onset
paramvaluesgen=paramvaluesgen.astype('float') # Only works with float values apparently

ttest=[]
tpvalues=[]
means=[]
variances=[]
importantvalues=[]
for i in varrange:
    delayedsubindex=subindex+i
    paramimportant=[]
    for i in delayedsubindex:
        paramimportant.append(paramvalues[i])
    importantvalues.append(paramimportant)
    paramimportant=np.array(paramimportant) # The test requires this apparently
    ttestvalues=stats.ttest_ind(a=paramvaluesgen[~np.isnan(paramvaluesgen)],b=paramimportant[~np.isnan(paramimportant)],equal_var=False) # The variations are NOT equal, so this is a Welch Test
    ttest.append(np.abs(ttestvalues.statistic))
    tpvalues.append(ttestvalues.pvalue)
    means.append(np.nanmean(paramimportant))
    variances.append(np.nanvar(paramimportant))

print("--- Time To Calculate: "+str(time.time()-starttime)+" seconds ("+str((time.time()-starttime)/60)+" minutes) ---")
print('Percent of substorms that fit the range criteria: '+str(len(subindex)/len(subtrange)*100)+' %')

starttime=time.time()
#np.savez(datadirectory+str(filename)+'start'+str(start)+'end'+str(end)+'nointerval'+'processeddata.npz',ttest,tpvalues,means,variances,genmean,genvariance,kstest,kspvalues,cvtest,cvpvalues,rstest,rspvalues,mwtest,mwpvalues)
# Comment out below if including the above terms
np.savez(datadirectory+str(filename)+'start'+str(start)+'end'+str(end)+'nointerval'+str(substormcutoff)+str(averagestart)+str(averageend)+'processeddata.npz',ttest,tpvalues,means,variances,genmean,genvariance,importantvalues)
#np.savez(datadirectory+str(filename)+'start'+str(start)+'end'+str(end)+'nointerval'+str(substormcutoff)+str(averagestart)+str(averageend)+'processeddata.npz',ttest,tpvalues,means,variances,genmean,genvariance)
print("--- Time To Save File: "+str(time.time()-starttime)+" seconds ("+str((time.time()-starttime)/60)+" minutes) ---")

22-Oct-24 18:49:08: C:\Users\getma\AppData\Local\Temp\ipykernel_12620\2540641442.py:29: RuntimeWarning: Mean of empty slice
  paramvaluesgen=np.nanmean(paramvaluesgen.reshape(-1,(abs(averageend-averagestart)+1)),axis=1) # Takes the average of each time period before a substorm onset



--- Time To Calculate: 0.15299773216247559 seconds (0.0025499622027079266 minutes) ---
Percent of substorms that fit the range criteria: 20.44747772003423 %
--- Time To Save File: 0.03599953651428223 seconds (0.0005999922752380372 minutes) ---


In [10]:
whos

Variable                     Type             Data/Info
-------------------------------------------------------
Pool                         method           <bound method BaseContext<...>t at 0x00000114737DF410>>
averageend                   int              5
averagestart                 int              -5
bxtimes                      ndarray          15252480: 15252480 elems, type `float64`, 122019840 bytes (116.3671875 Mb)
bxvalues                     ndarray          15252480: 15252480 elems, type `float32`, 61009920 bytes (58.18359375 Mb)
bygsmvalues                  ndarray          15252480: 15252480 elems, type `float32`, 61009920 bytes (58.18359375 Mb)
byvalues                     ndarray          15252480: 15252480 elems, type `float32`, 61009920 bytes (58.18359375 Mb)
bzgsmvalues                  ndarray          15252480: 15252480 elems, type `float32`, 61009920 bytes (58.18359375 Mb)
bzvalues                     ndarray          15252480: 15252480 elems, type `float32`, 